In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0


## 5.1.1 two dimentional cross-correlation

In [2]:
def corr2d(X, K):
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w +1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j].assign(tf.cast(tf.reduce_sum(X[i:i+h, j:j+w] * K), dtype=tf.float32))
    return Y

In [3]:
X = tf.constant([[0,1,2], [3,4,5], [6,7,8]])
K = tf.constant([[0,1], [2,3]])
corr2d(X, K)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[19., 25.],
       [37., 43.]], dtype=float32)>

## 5.1.2 Conv2d

In [4]:
class Conv2D(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
    
    def build(self, kernel_size):
        self.w = self.add_weight(name='w',
                                shape=kernel_size,
                                initializer=tf.random_normal_initializer())
        self.b = self.add_weight(name='b',
                                shape=(1,),
                                initializer=tf.random_normal_initializer())
    def call(self, inputs):
        return corr2d(inputs, self.w + self.b)

## 5.1.3 edge detection

In [5]:
X = tf.Variable(tf.ones((6,8)))
X[:, 2:6].assign(tf.zeros(X[:,2:6].shape))
X

<tf.Variable 'Variable:0' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

In [6]:
K = tf.constant([[1,-1]], dtype = tf.float32)
Y = corr2d(X, K)
Y

<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>

## 5.1.4 learn kernel by data

In [7]:
X = tf.reshape(X, (1,6,8,1))
Y = tf.reshape(Y, (1,6,7,1))
Y

<tf.Tensor: id=650, shape=(1, 6, 7, 1), dtype=float32, numpy=
array([[[[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]]]], dtype=float32)>

In [8]:
conv2d = tf.keras.layers.Conv2D(1, (1,2))
#input_shape = (samples, rows, cols, channels)
# Y = conv2d(X)
Y.shape

TensorShape([1, 6, 7, 1])

## debugging

In [9]:
Y_hat = conv2d(X)
for i in range(10):
    with tf.GradientTape(watch_accessed_variables=False) as g:
        g.watch(conv2d.variables)
        Y_hat = conv2d(X)
        l = (Y_hat - Y) ** 2
        dl = g.gradient(l, conv2d.variables)
#         print([conv2d.get_weights()[i] - dl[i] for i in range(len(dl))])
        conv2d.set_weights([conv2d.get_weights()[i] - 3e-2* dl[i] for i in range(len(dl))])
conv2d.get_weights()

[array([[[[828.0265 ]],
 
         [[826.07434]]]], dtype=float32), array([1477.3596], dtype=float32)]